# How to import VMAP meshes
This notebook demonstrates how to import finite element meshes in VMAP format in pylife, e.g., to be used for a FKM nonlinear assessment.

In [ ]:
# standard modules
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import timeit

# pylife
import pylife
import pylife.vmap
import pylife.mesh
import pylife.stress.equistress
import pylife.mesh.gradient

In [ ]:
# import the vmap file
filename = 'data/kt1.vmap'
vmap_mesh = pylife.vmap.VMAPImport(filename)

# select the geometry and state in the vmap file
geometry = 'KT1-1'
state = 'Step-1'
pylife_mesh = (vmap_mesh.make_mesh(geometry, state) 
               .join_coordinates()
               .join_variable('STRESS_CAUCHY')
               .to_frame())

# calculate the Mises equistress
pylife_mesh['mises'] = pylife_mesh.equistress.mises()

In [ ]:
pylife_mesh

We can filter only element Id 1:

In [ ]:
# entries of element 1
pylife_mesh[pylife_mesh.index.get_level_values("element_id") == 1]

In [ ]:
# Calculate the stress gradient
tstart = timeit.default_timer()
grad = pylife_mesh.gradient_3D.gradient_of('mises')
tend = timeit.default_timer()
print(f"duration calculate stress gradient: {tend-tstart:.1f} s")

grad["abs_grad"] = np.linalg.norm(grad, axis=1)
pylife_mesh = pylife_mesh.join(grad, sort=False)

In [ ]:
display(grad)

Next, we visualize the absolute stress gradient G using the `pyvista` package.

In [ ]:
### Plotting using pyvista
import pyvista as pv
grid = pv.UnstructuredGrid(*pylife_mesh.mesh.vtk_data())
plotter = pv.Plotter(window_size=[1920, 1080])
plotter.add_mesh(grid, scalars=pylife_mesh.groupby('element_id')['abs_grad'].mean().to_numpy(),
                show_edges=True, cmap='jet')
plotter.add_scalar_bar()
plotter.show()